# [모듈 0.0] pipeline.py 작성



# 1. src 폴더를 codemcommit 폴더에 카피

- 복사 소스
    - 3_MLOps/2_sm_pipeline/src
- 복사 위치
    - 3_MLOps/5_sm-serving-codepipeline/codecommit/pipelines/ncf/src

# 2. codebuild-buildspec.yml 파일
- 위치: 
    - codecommit/codebuild-buildspec.yml

```yml
version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
  
  build:
    commands:
      - python pipelines/upload_code.py --code-repository-name $code_repository_name --bucket $TEMPLATE_BUCKET
      - export PYTHONUNBUFFERED=TRUE
      - export SAGEMAKER_PROJECT_NAME_ID="${SAGEMAKER_PROJECT_NAME}-${SAGEMAKER_PROJECT_ID}"
      - |
        run-pipeline --module-name pipelines.ncf.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --kwargs "{\"region\":\"${AWS_REGION}\",\"sagemaker_project_arn\":\"${SAGEMAKER_PROJECT_ARN}\",\"role\":\"${SAGEMAKER_PIPELINE_ROLE_ARN}\",\"default_bucket\":\"${TEMPLATE_BUCKET}\",\"pipeline_name\":\"${SAGEMAKER_PIPELINE_NAME}\",\"model_package_group_name\":\"${SAGEMAKER_PIPELINE_NAME}\",\"s3_input_data_uri\":\"${s3_input_data_uri}\",\"project_prefix\":\"${project_prefix}\",\"base_job_prefix\":\"${SAGEMAKER_PIPELINE_NAME}\"}"
      - echo "Create/Update of the SageMaker Pipeline and execution completed."


```
- 이 파일에서 사용된 환경 변수는 codebuild_project 생성시에 환경 변수에서 가져오게 됨. 
     - 대표적으로 s3_input_data_uri 인 데이타 입력 경로 임. 이 값을 변경하기 위해서는 codebuild_project 의 환경 변수를 수정해야 함.

## 2.1 코드를 S3에 업로드

```
- python pipelines/upload_code.py --code-repository-name $code_repository_name --bucket $TEMPLATE_BUCKET
```      
- 위의 코드를 실행하면 code_location.json 파일이 생성 됨, 아래와 같이 코드가 압축이 되어서 S3에 source.tar.gz 파일이 저장 됨.
```
{
    "s3_location": "s3://sathiyajith"
}
```

## 2.2 파이프라인 실행

```Yml
run-pipeline --module-name pipelines.ncf.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --kwargs "{\"region\":\"${AWS_REGION}\",\"sagemaker_project_arn\":\"${SAGEMAKER_PROJECT_ARN}\",\"role\":\"${SAGEMAKER_PIPELINE_ROLE_ARN}\",\"default_bucket\":\"${ARTIFACT_BUCKET}\",\"pipeline_name\":\"${SAGEMAKER_PIPELINE_NAME}\",\"model_package_group_name\":\"${SAGEMAKER_PIPELINE_NAME}\",\"s3_input_data_uri\":\"${s3_input_data_uri}\",\"project_prefix\":\"${project_prefix}\",\"base_job_prefix\":\"${SAGEMAKER_PIPELINE_NAME}\"}"
```
- kwargs 이후의 파라미터는 파싱이 되어서 pipeline.py 의 get_pipeline() 함수의 파라미터로 전달이 됨.

### run-pipeline.py 의 실행 순서
- pipeline = _utils.get_pipeline_driver(args.kwargs)
    - get_pipeline(**kwargs)
- pipeline.upstart()    
- pipeline.start()


# 3. pipeline.py
- 위치
    - codecommit/pipelines/ncf/pipeline.py

## 3.1 get_pipeline(()

```
def get_pipeline(
    s3_input_data_uri,    
    project_prefix,
    region,
    sagemaker_project_arn=None,
    role=None, # SAGEMAKER_PIPELINE_ROLE_ARN 이 넘어옴.
    default_bucket=None,
    model_package_group_name="AbalonePackageGroup",
    pipeline_name="AbalonePipeline",
    base_job_prefix="Abalone",
    processing_instance_type="ml.m5.xlarge",
    training_instance_type="ml.p3.2xlarge",
):
```
- s3_input_data_uri
    - 데이타의 입력 경로 임.
- role
    - SAGEMAKER_PIPELINE_ROLE_ARN 경로 임. 
        - 이 노트북에서는 build_service role 을 사용함.

## 3.2  람다 스템을 위한 lambda_role 을 SAGEMAKER_PIPELINE_ROLE_ARN 로 대체 함.
- SAGEMAKER_PIPELINE_ROLE_ARN 은 아래와 같은 정책 및 신뢰 관계가 있어야 함.
- 필요 정책
    - AWSLambda_FullAccess
    - AmazonSageMakerFullAccess
- Trust Relationship    
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

## 3.3 람다 스텝 코드 위치 복사
- 아래와 같이 3_MLOps/3_sm-train-codepipeline/codecommit/pipelines/ncf 에 iam_repackage_model_artifact.py 복사

![iam_repackage_model_artifact.png](img/iam_repackage_model_artifact.png)

### 3.4 람다 스텝 변경
- script 에 repackage_lambda_script_path 의 절대 경로 지정

``` python
    repackage_lambda_script_path = f'{BASE_DIR}/iam_repackage_model_artifact.py'

    func_repackage_model = Lambda(
        function_name=function_name,
        execution_role_arn=role,
        script=repackage_lambda_script_path,        
        handler="iam_repackage_model_artifact.lambda_handler",
    )

```

## 3.5 Model Package Group Name 변경
- model_package_group_name 를 함수 인자로 받아서 사용 함.

```python
    model_package_group_input_dict = {
     "ModelPackageGroupName" : model_package_group_name,
     "ModelPackageGroupDescription" : "Sample model package group"
    }

```